<a href="https://colab.research.google.com/github/shreevatsdhyani/CL/blob/main/CLMovieRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages

In [1]:
!pip install requests openai python-dotenv --quiet

Add your API keys (Groq & TMDb)

In [2]:
import os

# Add your keys here or use dotenv
os.environ['GROQ_API_KEY'] = "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"
os.environ['TMDB_API_KEY'] = "c2edee2bf1f7ad6992b082e8265d6f96"


Initialize Groq client using the environment variable:

In [3]:
!pip install -q groq
# Import Groq client
from groq import Groq
import json
# Initialize Groq client with the API key from environment variable
groq_api_key = "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"
client = Groq(api_key=groq_api_key)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 6.6 MB/s eta 0:00:00


Validate the API Key (Optional but recommended)

In [4]:
import requests

# Retrieve the API key from environment variables
api_key = os.getenv('TMDB_API_KEY')

# Test API call to check if the key is working
test_url = f"https://api.themoviedb.org/3/movie/popular?api_key={api_key}"
test_response = requests.get(test_url).json()

# Check if the response contains a valid result
if test_response.get('status_code') == 7:
    print("API Key issue. Please check your key.")
else:
    print("API Key is working fine!")
    print(test_response)


API Key is working fine!
{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/fTrQsdMS2MUw00RnzH0r3JWHhts.jpg', 'genre_ids': [28, 80, 53], 'id': 1197306, 'original_language': 'en', 'original_title': 'A Working Man', 'overview': "Levon Cade left behind a decorated military career in the black ops to live a simple life working construction. But when his boss's daughter, who is like family to him, is taken by human traffickers, his search to bring her home uncovers a world of corruption far greater than he ever could have imagined.", 'popularity': 1092.1175, 'poster_path': '/xUkUZ8eOnrOnnJAfusZUqKYZiDu.jpg', 'release_date': '2025-03-26', 'title': 'A Working Man', 'video': False, 'vote_average': 6.4, 'vote_count': 375}, {'adult': False, 'backdrop_path': '/op3qmNhvwEvyT7UFyPbIfQmKriB.jpg', 'genre_ids': [14, 12, 28], 'id': 324544, 'original_language': 'en', 'original_title': 'In the Lost Lands', 'overview': 'A queen sends the powerful and feared sorceress Gray Alys to the ghostly wild

TMDb API - Search movies by filters (rating, genre, actor)

In [5]:
import requests
import os
def search_movies(query=None, genre=None, actor=None, min_rating=0, year_range=None, keywords=None):
    api_key = os.getenv("TMDB_API_KEY")
    base_url = "https://api.themoviedb.org/3"

    # Step 1: Get genre ID if genre is specified
    genre_id = None
    if genre:
        genre_response = requests.get(f"{base_url}/genre/movie/list", params={"api_key": api_key}).json()
        genre_dict = {g["name"].lower(): g["id"] for g in genre_response["genres"]}
        genre_id = genre_dict.get(genre.lower())

    # Step 2: Get actor ID if actor is specified
    actor_id = None
    if actor:
        actor_search = requests.get(f"{base_url}/search/person", params={"api_key": api_key, "query": actor}).json()
        if actor_search.get("results"):
            actor_id = actor_search["results"][0]["id"]

    # Step 3: Use DISCOVER endpoint with filters
    params = {
        "api_key": api_key,
        "language": "en-US",
        "include_adult": False,
        "sort_by": "vote_average.desc",  # Sort by rating (highest first)
        "vote_count.gte": 1000,  # Only include well-rated movies
    }

    if genre_id:
        params["with_genres"] = genre_id
    if actor_id:
        params["with_people"] = actor_id
    if min_rating > 0:
        params["vote_average.gte"] = min_rating
    if year_range:
        start_year, end_year = year_range
        params["primary_release_date.gte"] = f"{start_year}-01-01"
        params["primary_release_date.lte"] = f"{end_year}-12-31"

    discover_url = f"{base_url}/discover/movie"
    discover_response = requests.get(discover_url, params=params).json()
    movies = discover_response.get("results", [])

    # Filter out movies where romance isn't the primary genre
    if genre and genre.lower() == "romance":
        movies = [m for m in movies if "Romance" in [g["name"] for g in m.get("genres", [])]]

    # Fallback: If no movies, relax genre filter slightly
    if not movies and genre_id:
        params.pop("with_genres", None)
        discover_response = requests.get(discover_url, params=params).json()
        movies = discover_response.get("results", [])

    # Filter by keywords if provided
    if keywords:
        keyword_str = " ".join(keywords).lower()
        movies = [m for m in movies if keyword_str in m.get("overview", "").lower()]

    # Prioritize English/Hindi movies
    preferred_languages = ["en", "hi"]
    preferred = [m for m in movies if m.get("original_language") in preferred_languages]
    others = [m for m in movies if m.get("original_language") not in preferred_languages]

    return (preferred + others)[:5]  # Return top 5 results

Memory for Last 3 Conversations

In [6]:
conversation_memory = []

def update_memory(user_input, bot_response):
    conversation_memory.append((user_input, bot_response))
    if len(conversation_memory) > 3:
        conversation_memory.pop(0)

def get_memory_context():
    return "\n".join([f"User: {u}\nBot: {b}" for u, b in conversation_memory])


Extract Filters from Prompt using Groq

In [7]:
import openai
import json

def extract_filters(prompt):
    extract_prompt = f"""
Extract the movie filters in JSON format from the prompt below.

PROMPT: "{prompt}"

Return JSON with keys: genre, actor, year, rating.
Only include keys present in the prompt.

EXAMPLE OUTPUTS:
{{"genre": "romantic", "actor": "Emma Stone", "year": 2010, "rating": 7}}
{{"genre": "action"}}
{{"actor": "Tom Hanks", "rating": 8}}

Strictly output valid JSON. No extra text.
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": extract_prompt}],
        temperature=0.2
    )

    output_text = response.choices[0].message.content.strip()

    # Safely try to parse JSON
    try:
        filters = json.loads(output_text)
        return filters
    except Exception as e:
        print(f"⚠️ Filter parsing error: {e}\nRaw output: {output_text}")
        return {}  # Return empty dict to gracefully handle the failure


Generate Reply using Groq

In [8]:
def generate_reply(user_prompt, recommended_movies):
    memory = get_memory_context()
    movie_text = "\n".join([
        f"{m['title']} ({m['release_date'][:4] if m.get('release_date') else 'N/A'}) - Rating: {m.get('vote_average', 0)}"
        for m in recommended_movies
    ])

    full_prompt = f"""
You are a friendly movie assistant. Help the user with great movie suggestions USING EMOJIS to make it fun and engaging.

Memory:
{memory}

User asked: {user_prompt}

You found these movies:
{movie_text}

Rules:
- Use 2-3 emojis per movie suggestion (e.g., 🎬, ❤️, 🍿)
- Add genre/theme emojis (e.g., 🚀 for sci-fi, 🧟 for horror)
- Include rating stars (⭐)
- Keep it natural and not overdone!

Give a helpful reply using this info.
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",  # Updated model (or "mixtral-8x7b-32768" if preferred)
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.7
    )
    reply = response.choices[0].message.content
    update_memory(user_prompt, reply)
    return reply

Chat Function

In [9]:
def chat_with_bot(prompt):
    filters = extract_filters(prompt)
    genre = filters.get("genre")
    actor = filters.get("actor")
    year = filters.get("year")
    year_range = (year, 2025) if isinstance(year, int) else None
    min_rating = filters.get("rating", 0)

    keywords = filters.get("keywords", [])

    movies = search_movies(
        query=prompt,
        genre=genre,
        actor=actor,
        min_rating=min_rating,
        year_range=year_range,
        keywords=keywords
    )

    if not movies:
        return "No matching movies found. Try refining your preferences!"

    reply = generate_reply(prompt, movies)
    return reply


Test It

In [11]:
user_input = "suggest some movies like harry potter"

print(chat_with_bot(user_input))


Hey there, magic lover! ✨️🧙‍♂️

You're looking for movies like Harry Potter, eh? ⚡️ I've got some enchanting suggestions for you! 😉

Since you enjoy fantasy adventures with a touch of magic, here are some spellbinding recommendations:

**The Green Mile (1999) - ⭐⭐⭐⭐⭐ 8.504** ⚡️👮‍♂️
A fantastical tale of friendship, hope, and redemption, set in a prison with a touch of the supernatural.

**The Lord of the Rings Trilogy (2001-2003) - ⭐⭐⭐⭐⭐ 8.42-8.483** ⚔️🔥🏰
Experience the epic fantasy adventure of a lifetime, with stunning battles, unforgettable characters, and a quest to save Middle-earth!

**It's a Wonderful Life (1946) - ⭐⭐⭐⭐⭐ 8.277** ❤️🎄
A heartwarming classic about the power of kindness, friendship, and the magic of the human spirit.

Hope you find something that casts a spell on you! 😊 Let me know if you need more recommendations.
